In [1]:
import mlflow
import time
import mlflow.system_metrics
from mlflow.models import EvaluationMetric

import xgboost
import shap
import mlflow
from sklearn.model_selection import train_test_split
import matplotlib.pyplot as plt
import numpy as np

/workspace/.venv/lib/python3.11/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [5]:
mlflow.set_tracking_uri(
    "http://ec2-44-217-145-52.compute-1.amazonaws.com:5000"
)
experiment = mlflow.set_experiment("test")
mlflow.system_metrics.enable_system_metrics_logging()

In [6]:
run = mlflow.start_run(experiment_id=experiment.experiment_id)

2024/10/28 14:39:48 INFO mlflow.system_metrics.system_metrics_monitor: Started monitoring system metrics.


In [7]:
mlflow.log_metrics({"tlearner_auuc": 0.1, "tlearner_uplift": 0.01})
time.sleep(10)
mlflow.log_metrics({"slearner_auuc": 0.2, "tlearner_uplift": 0.02})

In [8]:
# load UCI Adult Data Set; segment it into training and test sets
X, y = shap.datasets.adult()
X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.33, random_state=42
)

# train XGBoost model
model = xgboost.XGBClassifier().fit(X_train, y_train)

# construct an evaluation dataset from the test set
eval_data = X_test
eval_data["target"] = y_test

In [9]:
# load UCI Adult Data Set; segment it into training and test sets
X, y = shap.datasets.adult()
X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.33, random_state=42
)

# train XGBoost model
model = xgboost.XGBClassifier().fit(X_train, y_train)

# construct an evaluation dataset from the test set
eval_data = X_test
eval_data["target"] = y_test

In [31]:
def example_custom_metric_fn(predictions, target, metrics):
    return np.sum(np.abs(predictions - target + 1) ** 2)


def example_custom_artifact_fn(predictions, _builtin_metrics, _artifacts_dir):
    fig = plt.figure()
    ax = fig.add_subplot(111)
    ax.scatter(predictions["prediction"], predictions["target"])
    ax.set_xlabel("Targets")
    ax.set_ylabel("Predictions")
    ax.set_title("Targets vs. Predictions")
    return {"example_scatter_plot_artifact": fig}


def pred_sample(predictions, _builtin_metrics, _artifacts_dir):
    return {"pred_sample": predictions.head(10)}


custom_metric = EvaluationMetric(
    example_custom_metric_fn,
    "example_custom_metric_fn",
    greater_is_better=True,
    long_name="Example Custom Metric",
)

In [49]:
len(eval_data.columns)

14

In [50]:
eval_data["tg_cg"] = 1

model_info = mlflow.sklearn.log_model(model, "model")
result = mlflow.evaluate(
    model_info.model_uri,
    eval_data.to_numpy(),
    targets="target",
    predictions="tgcg",
    # feature_names=[col for col in eval_data.columns if col not in ["target", "tg_cg"]],
    extra_metrics=[custom_metric],
    custom_artifacts=[example_custom_artifact_fn, pred_sample],
)

2024/10/28 15:43:03 WARNING mlflow.models.model: Model logged without a signature and input example. Please set `input_example` parameter when logging the model to auto infer the model signature.
2024/10/28 15:43:03 WARNING mlflow.models.model: Logging model metadata to the tracking server has failed. The model artifacts have been logged successfully under s3://kaggle-nakayama/mlflow/1/296477b32de74b97874030fd124b1763/artifacts. Set logging level to DEBUG via `logging.getLogger("mlflow").setLevel(logging.DEBUG)` to see the full traceback.


AttributeError: 'list' object has no attribute 'flatten'

In [9]:
# mlflow.end_run()

In [33]:
eval_data

,Age,Workclass,Education-Num,Marital Status,Occupation,Relationship,Race,Sex,Capital Gain,Capital Loss,Hours per week,Country,target
14160,27.0,4,10.0,0,1,0,4,0,0.0,0.0,38.0,39,False
27048,45.0,7,9.0,2,4,5,4,0,0.0,0.0,40.0,39,False
28868,29.0,4,13.0,2,4,4,2,1,0.0,0.0,55.0,39,True
5667,30.0,4,13.0,4,7,0,4,0,0.0,0.0,40.0,39,False
7827,29.0,6,10.0,0,3,0,4,1,2202.0,0.0,50.0,39,False
...,...,...,...,...,...,...,...,...,...,...,...,...,...
12795,22.0,4,10.0,4,11,3,1,1,0.0,0.0,24.0,19,False
12129,33.0,4,9.0,4,14,2,1,1,0.0,0.0,40.0,39,False
12017,65.0,2,9.0,2,3,4,4,1,0.0,0.0,40.0,39,False
22468,42.0,6,9.0,2,12,4,4,1,0.0,0.0,60.0,39,False
